<a href="https://colab.research.google.com/github/Sirfowahid/FFN/blob/master/FNN_Optimizers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset,DataLoader
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from IPython import display
display.set_matplotlib_formats('svg') 

In [4]:
data = np.loadtxt(open('sample_data/mnist_train_small.csv','rb'),delimiter=',')

labels = data[:,0]
data = data[:,1:]

dataNorm = data/np.max(data)

In [5]:
dataT = torch.tensor(dataNorm).float()
labelsT = torch.tensor(labels).long()

train_data,test_data,train_labels,test_labels = train_test_split(dataT,labelsT,test_size=.1)

train_dataset = TensorDataset(train_data,train_labels)
test_dataset = TensorDataset(test_data,test_labels)

batchsize = 32
train_loader = DataLoader(train_dataset,batch_size=batchsize,shuffle=True,drop_last=True)
test_loader = DataLoader(test_dataset,batch_size=test_dataset.tensors[0].shape[0])

In [6]:
from torch.optim import optimizer
def createTheMNISTNet(optimizerAlgo,learningrate):

  class mnistnet(nn.Module):

    def __init__(self):
      super().__init__()

      self.input = nn.Linear(784,64)

      self.fc1 = nn.Linear(64,32)
      self.fc2 = nn.Linear(32,32)

      self.output = nn.Linear(32,10)

    def forward(self,x):

      x = F.relu(self.input(x))
      x = F.relu(self.fc1(x))
      x = F.relu(self.fc2(x))

      return self.output(x)
  
  net = mnistnet()

  lossfun = nn.CrossEntropyLoss()

  optifun = getattr(torch.optim,optimizerAlgo)
  optimizer = optifun(net.parameters(),lr = learningrate)

  return net,lossfun,optimizer

In [ ]:
def func2trainTheModel(optimizerAlgo,learningrate):

  numepochs = 100

  net,lossfun,optimizer = createTheMNISTNet(optimizerAlgo,learningrate)

  losses = torch.zeros(numepochs)
  trainAcc = []
  testAcc = []

  for epochi in range(numepochs):

    batchAcc = []
    batchLoss = []

    for X,y in train_loader:

      yHat = net(X)
      loss = lossfun(yHat,y)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      batchLoss.append(loss.item())

      matches = torch.argmax(yHat,axis=1)==y
      matchesNumeric = matches.float()
      accPct = 100*torch.mean(matchesNumeric)
      batchAcc.append(accPct)

    trainAcc.append(np.mean(batchAcc))
    losses[epochi] = np.mean(batchLoss)

    X,y = next(iter(test_loader))
    with torch.no_grad():
      yHat = net(X)

    testAcc.append(100*torch.mean((torch.argmax(yHat,axis=1)==y).float()).item())
  
  return trainAcc,testAcc,losses,net
